Для любой модели из курса(кроме KNN) и данных на которых она строилась: применить метод главных компонент к данным, и сравнить работу модели на исходных данных с работой на данных где в качестве признаков используются две главные компоненты.

### PCA на адаптивном бустинге

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn import model_selection
import numpy as np

In [19]:
#загрузим данные
X, y = load_breast_cancer(return_X_y=True)

In [20]:
# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [21]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_, y, test_size=0.25)
def get_error(pred, y):
    return np.sum((pred != y).astype(int)) / len(y)

In [22]:
def adaboost(X, y, N):

    # Размер выборки
    n_objects = len(X)

    # Запишем количество классов в переменную
    n_classes = len(np.unique((y)))

    # Начальные веса деревьев
    w = np.ones(n_objects) / n_objects

    # Деревья с весами будем записывать в список
    models = []

    for n in range(N):
        # Зададим дерево и обучим его
        clf = DecisionTreeClassifier(max_depth=1)
        clf.fit(X, y, sample_weight=w)

        predictions = clf.predict(X)
        e = get_error(predictions, y)
        # отбросим дерево, если его ошибка больше 0.5
        # Запишем условие в общем виде (применимо к небинарным классификаторам)
        if e >= 1 - 1/n_classes: 
            break

        # Вычислим вес для дерева
        alpha = 0.5 * np.log((1 - e) / e)

        # Найдем индексы правильно классифицированных элементов
        match = predictions == y

        # Увеличим веса для неправильно классифицированных элементов
        w[~match] *= np.exp(alpha)

        # Нормализуем веса
        w /= w.sum()

        # Добавим дерево с весом в список
        models.append((alpha, clf))
    
    return models

In [23]:
# Обучающая выборка
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_train.T.dot(X_train)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(30,1), eig_pairs[1][1].reshape(30,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[-0.2188009   0.23362047]
 [-0.10744801  0.04572229]
 [-0.22772692  0.21527758]
 [-0.22318202  0.22894126]
 [-0.13128342 -0.19026572]
 [-0.23784618 -0.14784133]
 [-0.25846808 -0.06157384]
 [-0.26086378  0.03203132]
 [-0.1393337  -0.18544536]
 [-0.05606383 -0.37101427]
 [-0.21106771  0.0834839 ]
 [-0.01330533 -0.10973684]
 [-0.21857462  0.06743071]
 [-0.20873495  0.14023293]
 [ 0.00110711 -0.21912847]
 [-0.16348862 -0.22810227]
 [-0.15200011 -0.22142509]
 [-0.18339267 -0.16723088]
 [-0.03393853 -0.1643044 ]
 [-0.09326325 -0.29684264]
 [-0.22885645  0.21525896]
 [-0.1100144   0.03862446]
 [-0.23722901  0.19550848]
 [-0.22719434  0.21180958]
 [-0.12194186 -0.17755383]
 [-0.21040059 -0.12852955]
 [-0.22943996 -0.09140652]
 [-0.25217913  0.00679032]
 [-0.12209061 -0.10666067]
 [-0.1250982  -0.27484309]]


In [24]:
# Сформируем новую матрицу "объекты-признаки"
new_X_train = X_train.dot(W)

In [25]:
new_X_test = X_test.dot(W)

In [26]:
N = 50

models = adaboost(new_X_train, y_train, N)

In [27]:
def predict(X, models):
    
    n_classes = 2
    n_objects = len(X)
    
    # вначале обозначим предсказание нулевым массивом
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = clf.predict(X)
        # Для каждого предсказания будем прибавлять alpha к
        # элементу с индексом предсказанного класса
        y_pred[range(n_objects), prediction] += alpha
    
    # выберем индексы с максимальными суммарными весами -
    # получим предсказанные алгоритмом классы
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict(new_X_train, models), y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 91.080


In [28]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict(new_X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 96.503


In [ ]:
#Вывод: на обучающей выборке с PCA точность оказалась ниже, чем на обучающей выборке без PCA. 
#Но на тестовой выборке с PCA точность алгоритма оказалась выше, чем на тестовой выборке без PCA.